### Comparison Operators

We saw that we have two types of equality comparison:
- value comparison (`==`)
- object identity (`is`)

When we use the `==` operator, the type itself (an object with functionality and state), specifies how two different objects may be equal, in some sense.

For numeric types, the `==` comparison compares the **value** of the number.

So we have this:

In [1]:
a = 10
b = 10

In [2]:
a == b

True

And since `10.0` and `10` are both represented exactly in Python, even though they are different types, they will still compare equal:

In [3]:
c = 10.0

In [4]:
a == c

True

But `a` and `c` are not the same objects:

In [5]:
a is c

False

One way we can investigate objects and whether two objects are the same objects, is to look at their **memory address**.

Objects in Python reside somewhere in memory. Think of memory as a bunch of slots, each of which has an **address** - just like mailboxes have addresses.

If the memory address of an object is the same as the other one, then they are the same object - just like two addresses that are the same correspond to the same mailbox.

If two objects are identical (the same object), they are obviously equal too (two mailboxes that correspond to the same address **are** the same mailbox, so they must have the same content).

In Python we can use the `id` function to see the memory address of an object:

In [6]:
a = 10
b = 10.0

In [7]:
a == b

True

In [8]:
a is b

False

In [9]:
id(a), id(b)

(140708869720792, 2269555909040)

As you can see, `a` and `b` are not the same object (`a is b` evaluates to `False`), and indeed the memory address of the two objects are different. 

Related to the `==` operator, we have the `!=` (no equal) operator - again it works with values, not object identity:

In [10]:
10 != 12

True

In [11]:
10.5 != 10.5

False

We have other types of comparison operators, such as `<`, `<=`, etc.

These operators works very similarly to the `==` operator - the types themselves define what those operators mean.

Sometimes a type does not define that, in which case there is no ordering (called natural ordering).

Integers and floats are naturally ordered and Python implements that for us:

In [12]:
10 >= 5

True

In [13]:
10.5 < 100.2

True

Even across numeric types:

In [14]:
10 <= 12.5

True

These operators work because the `int` and `float` types implement special functionality (methods), such as `__lt__`, `__lte__`, etc. We will be able to do the same with our own custom types later in this course.

Not all types implement all these comparison operators. For example, complex numbers in Python do not (they implement equality, but not ordering):

In [15]:
a = 1 + 1j
b = 1 + 1j
c = 2 + 2j

In [16]:
a == b

True

In [17]:
a is b, id(a), id(b)

(False, 2269555909776, 2269555910704)

As we can see, `a` and `b` are two different objects (`a is b` evaluates to `False`, and indeed the memory addresses are different), but they are `==` in terms of their value.

On the other hand what does it mean for one complex number to be less than another one? Well that may depend on the context or how we want to define it - so Python makes no assumption, and simply does not implement order comparisons for complex numbers:

In [18]:
a < c

TypeError: '<' not supported between instances of 'complex' and 'complex'

And we simply get an exception.

#### Floats and Equality

We've mentioned this before, but because floats rarely have exact representations in Python, we should not use `==` to test if two float values are the same:

In [19]:
0.1 * 3 == 0.3

False

That's because the representations for `0.1` multiplied by `3`, is not exactly the same as the representation for `0.3` - very very close, but not exact.

In [20]:
format(0.1 * 3, '.25f')

'0.3000000000000000444089210'

In [21]:
format(0.3, '.25f')

'0.2999999999999999888977698'

One way around this problem is to define some tolerance, and if the absolute value of the difference of the two floats is less than the tolerance, we can accept the numbers as equal:

In [22]:
tol = 0.000_000_001

In [23]:
a = 0.1 * 3
b = 0.3

print(format(a, '.25f'))
print(format(b, '.25f'))
print(abs(a-b) < tol)

0.3000000000000000444089210
0.2999999999999999888977698
True


#### Custom Types

As before, I'm just going to show you something, but I don't expect you to understand all this code right now - I just want to show you that comparison operators such as `==`, `<`, `<=`, etc are actually defined by the type itself.

Let's use our `Vector` example from before:

In [24]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        return None

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'

In [25]:
v1 = Vector(1, 1)
v2 = Vector(1, 1)
v3 = Vector(2, 3)

In [26]:
id(v1), id(v2), id(v3)

(2269556121984, 2269556642048, 2269556651456)

As we can see, each of those vectors are different objects (different memory addresses).

So it should not surprise us that `v1` and `v2` will compare `False` when using the identity comparison:

In [27]:
v1 is v2

False

But, maybe not what we would expect is that from a equality comparison perspective, they also compare `False`:

In [28]:
v1 == v2

False

This happens because Python has no way of knowing how to compare the "value" of two `Vector` objects - so it falls back to looking at the identity (essentially falls back to using `is`).

Of course, we can tell Python how to perform an equality comparison, by adding that functionality to our objects:

In [28]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        return None

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return self.x == other.x and self.y == other.y
        return False

Doing this we now have defined for Python to handle `==` between two vectors:

In [29]:
v1 = Vector(1, 1)
v2 = Vector(1, 1)
v3 = Vector(2, 3)

In [30]:
v1 is v2

False

In [31]:
v1 == v2

True

And of course, `v1` and `v3` are neither the same object:

In [32]:
v1 is v3

False

nor do they have the same values:

In [33]:
v1 == v3

False

In the same way our vectors do not implement ordering:

In [34]:
v1 < v3

TypeError: '<' not supported between instances of 'Vector' and 'Vector'

Once again, we can tell Python how to handle this operator (`<`) by implementing that functionality in our objects using the special method `__lt__` (`l`ess `t`han):

Here I'll choose to define `v1 < v2` if the length of `v1` is less than the length of `v2`.

The length of a vector with components `(x, y)` is given by:

$$
\sqrt{x^2 + y^2}
$$

So if we have two vectors `v1 = (x1, y1)` and `v2 = (x2, y2)`, we will **define** `v1` to be less than `v2` if:

$$
\sqrt{x_1^2 + y_1^2} < \sqrt{x_2^2 + y_2^2}
$$

But we can remove the square roots on both sides, to get:

$$
x_1^2 + y_1^2 < x_2^2 + y_2^2
$$

Let's implement this for our vectors:

In [37]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __add__(self, other):
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        return None

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return self.x == other.x and self.y == other.y
        return False
    
    def __lt__(self, other):
        if isinstance(other, Vector):
            return (self.x ** 2 + self.y ** 2) < (other.x ** 2 + other.y ** 2)
        raise TypeError('< between Vector and non-Vector is not supported.')

In [38]:
v1 = Vector(1, 1)
v2 = Vector(1, 1)
v3 = Vector(2, 3)

In [39]:
v2 < v3

True

We can also see our custom exception if we try to compare a vector to something that is not a vector:

In [40]:
v2 < 100

TypeError: < between Vector and non-Vector is not supported.

We'll come back to this type of code later in this course.

Finally, I'm again to jump ahead a bit and talk about the `in` operator.

Python has many collection type objects: objects that contain other objects.

In Math, we have sets for example. Sets are one type of container type object in Python.

We can create a simple set using a literal notation as follows:

In [41]:
s = {1, 2, 3.14, 5, True}

As you can see, sets can contain heterogenous data (mixed data types).

We can test whether a value is in our set by using the `in` operator:

In [42]:
1 in s

True

In [43]:
3.14 in s

True

In [44]:
True in s

True

In [45]:
False in s

False

In [46]:
100 in s

False

We also have the `not in`operator (yes, it's two words, but it's actually a single binary operator):

In [47]:
True not in s

False

In [48]:
100 not in s

True

We'll come back to collection types later in this course.